In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_validate,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,classification_report

In [2]:
data=pd.read_csv("Desktop/churn.csv.txt")

In [3]:
data.head()

,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


In [4]:
data["Churn?"].replace(["False.","True."],[0,1],inplace=True)

In [5]:
data["Int'l Plan"].replace(["yes","no"],(1,0),inplace=True)
data["VMail Plan"].replace(["yes","no"],(1,0),inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State           3333 non-null   object 
 1   Account Length  3333 non-null   int64  
 2   Area Code       3333 non-null   int64  
 3   Phone           3333 non-null   object 
 4   Int'l Plan      3333 non-null   int64  
 5   VMail Plan      3333 non-null   int64  
 6   VMail Message   3333 non-null   int64  
 7   Day Mins        3333 non-null   float64
 8   Day Calls       3333 non-null   int64  
 9   Day Charge      3333 non-null   float64
 10  Eve Mins        3333 non-null   float64
 11  Eve Calls       3333 non-null   int64  
 12  Eve Charge      3333 non-null   float64
 13  Night Mins      3333 non-null   float64
 14  Night Calls     3333 non-null   int64  
 15  Night Charge    3333 non-null   float64
 16  Intl Mins       3333 non-null   float64
 17  Intl Calls      3333 non-null   i

In [7]:
data.drop(["State","Area Code","Phone"],axis=1,inplace=True)

In [8]:
target=data["Churn?"]
X=data.drop("Churn?",axis=1)

In [9]:
xtr,xte,ytr,yte=train_test_split(X,target,test_size=0.2,random_state=123)

In [10]:
tree_model=DecisionTreeClassifier(random_state=1)
fold=KFold(n_splits=5)

par_tree={'max_depth' : [8,10,12],
         'min_samples_leaf' : [5,10]}

In [11]:
grid_tree=GridSearchCV(tree_model,param_grid=par_tree,cv=fold,verbose=1)

In [12]:
grid_tree.fit(xtr,ytr)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(random_state=1),
             param_grid={'max_depth': [8, 10, 12], 'min_samples_leaf': [5, 10]},
             verbose=1)

In [13]:
grid_tree.best_estimator_

DecisionTreeClassifier(max_depth=10, min_samples_leaf=10, random_state=1)

In [14]:
grid_tree.cv_results_

{'mean_fit_time': array([0.03509235, 0.02540894, 0.04135394, 0.03375802, 0.04656167,
        0.04602394]),
 'std_fit_time': array([0.0122736 , 0.0031496 , 0.00806036, 0.00288858, 0.01043517,
        0.01473008]),
 'mean_score_time': array([0.00351338, 0.00361133, 0.00256991, 0.00167899, 0.00302386,
        0.00370259]),
 'std_score_time': array([0.00168354, 0.00258407, 0.00185454, 0.00145164, 0.00115008,
        0.00218121]),
 'param_max_depth': masked_array(data=[8, 8, 10, 10, 12, 12],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_leaf': masked_array(data=[5, 10, 5, 10, 5, 10],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 8, 'min_samples_leaf': 5},
  {'max_depth': 8, 'min_samples_leaf': 10},
  {'max_depth': 10, 'min_samples_leaf': 5},
  {'max_depth': 10, 'min_samples_leaf': 10},
  {'max_depth': 12, 'min

In [15]:
pred_tree=grid_tree.predict(xte)

In [16]:
print(classification_report(yte,pred_tree))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96       572
           1       0.79      0.74      0.76        95

    accuracy                           0.93       667
   macro avg       0.87      0.85      0.86       667
weighted avg       0.93      0.93      0.93       667



In [18]:
rf_model=RandomForestClassifier()
para_rf={"n_estimators":[25,52,75],
        "max_depth":[8,10,12],
        "max_features":[4,5,6,7]}

In [19]:
grid_rf=GridSearchCV(rf_model,param_grid=para_rf,cv=fold,verbose=1)

In [20]:
grid_rf.fit(xtr,ytr)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(),
             param_grid={'max_depth': [8, 10, 12], 'max_features': [4, 5, 6, 7],
                         'n_estimators': [25, 52, 75]},
             verbose=1)

In [21]:
grid_rf.best_estimator_

RandomForestClassifier(max_depth=8, max_features=7, n_estimators=75)

In [23]:
print("mean of accuracy of cv:",grid_rf.cv_results_["mean_test_score"].mean())
print("mean of std of cv:",grid_rf.cv_results_["std_test_score"].mean())

mean of accuracy of cv: 0.9487532001508432
mean of std of cv: 0.009479812057928295


In [24]:
pred_rf=grid_rf.predict(xte)

In [26]:
print(classification_report(yte,pred_rf))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97       572
           1       0.85      0.73      0.78        95

    accuracy                           0.94       667
   macro avg       0.90      0.85      0.88       667
weighted avg       0.94      0.94      0.94       667



In [27]:
#we can use balanced in class_weight to solve imbalanced tree

In [28]:
grid_gbt=GradientBoostingClassifier()
para_gbt={"n_estimators":[50,100,150],
         "learning_rate":[0.1,0.05,0.01],
         "max_depth":[3,4,5]}

In [29]:
grid_gbt=GridSearchCV(grid_gbt,param_grid=para_gbt,cv=fold,verbose=1)

In [30]:
grid_gbt.fit(xtr,ytr)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': [0.1, 0.05, 0.01],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [50, 100, 150]},
             verbose=1)

In [32]:
grid_gbt.best_estimator_

GradientBoostingClassifier(max_depth=5, n_estimators=150)

In [33]:
print("mean of accuracy of cv:",grid_gbt.cv_results_["mean_test_score"].mean())
print("mean of std of cv:",grid_gbt.cv_results_["std_test_score"].mean())

mean of accuracy of cv: 0.9330053349510735
mean of std of cv: 0.011418176142988524


In [38]:
!pip install xgboost

     -------------------------------------- 70.9/70.9 MB 997.7 kB/s eta 0:00:00


In [44]:
import xgboost as xgb
xgc=xgb.XGBClassifier( objective='binary:logistic')

In [54]:
para_xgc={
         "n_estimators":[50,75,150],
         "eta":[0.1,0.05],
         "max_depth":[4,5]}
#eta is learning_rate in xgboost

In [55]:
grid_xgc=GridSearchCV(xgc,param_grid=para_xgc,cv=fold)

In [58]:
grid_xgc.fit(xtr,ytr,eval_set=[(xtr,ytr),(xte,yte)],
            eval_metric=['logloss','error'])

[0]	validation_0-logloss:0.62188	validation_0-error:0.07277	validation_1-logloss:0.62393	validation_1-error:0.08546
[1]	validation_0-logloss:0.56324	validation_0-error:0.05889	validation_1-logloss:0.56660	validation_1-error:0.06897
[2]	validation_0-logloss:0.51397	validation_0-error:0.06977	validation_1-logloss:0.51860	validation_1-error:0.08846
[3]	validation_0-logloss:0.47256	validation_0-error:0.05851	validation_1-logloss:0.47799	validation_1-error:0.06897
[4]	validation_0-logloss:0.43700	validation_0-error:0.05851	validation_1-logloss:0.44342	validation_1-error:0.06897
[5]	validation_0-logloss:0.40644	validation_0-error:0.05664	validation_1-logloss:0.41402	validation_1-error:0.07046
[6]	validation_0-logloss:0.37992	validation_0-error:0.05664	validation_1-logloss:0.38928	validation_1-error:0.07046
[7]	validation_0-logloss:0.35691	validation_0-error:0.05664	validation_1-logloss:0.36640	validation_1-error:0.07046
[8]	validation_0-logloss:0.33442	validation_0-error:0.05626	validation_1

D:\python\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[19]	validation_0-logloss:0.20245	validation_0-error:0.03713	validation_1-logloss:0.21410	validation_1-error:0.04648
[20]	validation_0-logloss:0.19661	validation_0-error:0.03863	validation_1-logloss:0.20888	validation_1-error:0.04498
[21]	validation_0-logloss:0.19040	validation_0-error:0.03788	validation_1-logloss:0.20121	validation_1-error:0.04348
[22]	validation_0-logloss:0.18601	validation_0-error:0.03601	validation_1-logloss:0.19733	validation_1-error:0.04348
[23]	validation_0-logloss:0.18178	validation_0-error:0.03563	validation_1-logloss:0.19338	validation_1-error:0.04198
[24]	validation_0-logloss:0.17735	validation_0-error:0.03563	validation_1-logloss:0.18773	validation_1-error:0.04048
[25]	validation_0-logloss:0.17340	validation_0-error:0.03526	validation_1-logloss:0.18426	validation_1-error:0.03748
[26]	validation_0-logloss:0.16934	validation_0-error:0.03526	validation_1-logloss:0.17993	validation_1-error:0.03748
[27]	validation_0-logloss:0.16617	validation_0-error:0.03488	val

D:\python\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[21]	validation_0-logloss:0.19089	validation_0-error:0.03788	validation_1-logloss:0.19991	validation_1-error:0.04498
[22]	validation_0-logloss:0.18610	validation_0-error:0.03788	validation_1-logloss:0.19591	validation_1-error:0.04648
[23]	validation_0-logloss:0.18127	validation_0-error:0.03826	validation_1-logloss:0.19009	validation_1-error:0.04498
[24]	validation_0-logloss:0.17752	validation_0-error:0.03788	validation_1-logloss:0.18696	validation_1-error:0.04648
[25]	validation_0-logloss:0.17381	validation_0-error:0.03638	validation_1-logloss:0.18253	validation_1-error:0.04498
[26]	validation_0-logloss:0.17094	validation_0-error:0.03638	validation_1-logloss:0.17982	validation_1-error:0.04498
[27]	validation_0-logloss:0.16788	validation_0-error:0.03676	validation_1-logloss:0.17658	validation_1-error:0.04498
[28]	validation_0-logloss:0.16494	validation_0-error:0.03563	validation_1-logloss:0.17437	validation_1-error:0.04198
[29]	validation_0-logloss:0.16231	validation_0-error:0.03563	val

...

[4]	validation_0-logloss:0.43489	validation_0-error:0.05289	validation_1-logloss:0.44077	validation_1-error:0.06147
[5]	validation_0-logloss:0.40418	validation_0-error:0.05364	validation_1-logloss:0.41133	validation_1-error:0.06297
[6]	validation_0-logloss:0.37724	validation_0-error:0.05364	validation_1-logloss:0.38542	validation_1-error:0.06597
[7]	validation_0-logloss:0.35215	validation_0-error:0.05326	validation_1-logloss:0.35991	validation_1-error:0.06897
[8]	validation_0-logloss:0.33005	validation_0-error:0.05176	validation_1-logloss:0.33820	validation_1-error:0.06597
[9]	validation_0-logloss:0.30966	validation_0-error:0.05251	validation_1-logloss:0.31835	validation_1-error:0.06897
[10]	validation_0-logloss:0.29205	validation_0-error:0.05214	validation_1-logloss:0.30160	validation_1-error:0.07046
[11]	validation_0-logloss:0.27648	validation_0-error:0.04914	validation_1-logloss:0.28650	validation_1-error:0.06447
[12]	validation_0-logloss:0.26328	validation_0-error:0.04651	validatio

D:\python\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[17]	validation_0-logloss:0.21283	validation_0-error:0.03713	validation_1-logloss:0.22738	validation_1-error:0.05997
[18]	validation_0-logloss:0.20549	validation_0-error:0.03676	validation_1-logloss:0.22016	validation_1-error:0.05997
[19]	validation_0-logloss:0.19899	validation_0-error:0.03601	validation_1-logloss:0.21434	validation_1-error:0.05547
[20]	validation_0-logloss:0.19334	validation_0-error:0.03676	validation_1-logloss:0.20910	validation_1-error:0.05697
[21]	validation_0-logloss:0.18824	validation_0-error:0.03601	validation_1-logloss:0.20504	validation_1-error:0.05697
[22]	validation_0-logloss:0.18245	validation_0-error:0.03526	validation_1-logloss:0.19860	validation_1-error:0.05547
[23]	validation_0-logloss:0.17848	validation_0-error:0.03601	validation_1-logloss:0.19448	validation_1-error:0.05547
[24]	validation_0-logloss:0.17379	validation_0-error:0.03488	validation_1-logloss:0.18936	validation_1-error:0.05247
[25]	validation_0-logloss:0.16997	validation_0-error:0.03451	val

[88]	validation_0-logloss:0.10794	validation_0-error:0.02363	validation_1-logloss:0.15857	validation_1-error:0.04948
[89]	validation_0-logloss:0.10758	validation_0-error:0.02326	validation_1-logloss:0.15841	validation_1-error:0.04948
[90]	validation_0-logloss:0.10728	validation_0-error:0.02326	validation_1-logloss:0.15812	validation_1-error:0.04798
[91]	validation_0-logloss:0.10686	validation_0-error:0.02326	validation_1-logloss:0.15842	validation_1-error:0.04948
[92]	validation_0-logloss:0.10661	validation_0-error:0.02326	validation_1-logloss:0.15845	validation_1-error:0.04948
[93]	validation_0-logloss:0.10621	validation_0-error:0.02326	validation_1-logloss:0.15846	validation_1-error:0.04798
[94]	validation_0-logloss:0.10545	validation_0-error:0.02288	validation_1-logloss:0.15837	validation_1-error:0.04648
[95]	validation_0-logloss:0.10505	validation_0-error:0.02288	validation_1-logloss:0.15865	validation_1-error:0.04498
[96]	validation_0-logloss:0.10460	validation_0-error:0.02251	val

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,...
                                     learning_rate=None, max_bin=None,
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, 

In [57]:
grid_xgc.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.1, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=150, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [52]:
print("mean of accuracy of cv:",grid_xgc.cv_results_["mean_test_score"].mean())
print("mean of std of cv:",grid_xgc.cv_results_["std_test_score"].mean())

mean of accuracy of cv: 0.9523578641145098
mean of std of cv: 0.008461508733120834


In [59]:
pred_xgc=grid_xgc.predict(xte)
print(classification_report(yte,pred_xgc))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       572
           1       0.90      0.80      0.85        95

    accuracy                           0.96       667
   macro avg       0.94      0.89      0.91       667
weighted avg       0.96      0.96      0.96       667

